In [162]:
!pip3 install pyvespa

Defaulting to user installation because normal site-packages is not writeable


In [29]:
from vespa.package import (
    ApplicationPackage,
    Component,
    Parameter,
    Field,
    HNSW,
    RankProfile,
    Function,
    FirstPhaseRanking,
    SecondPhaseRanking,
    FieldSet,
    DocumentSummary,
    Summary,
)
from pathlib import Path
import json

app_package = ApplicationPackage(
    name="paragraph",
    components=[
        Component(
            id="e5-small-q",
            type="hugging-face-embedder",
            parameters=[
                Parameter("transformer-model", {"path": "model/e5-small-v2-int8.onnx"}),
                Parameter("tokenizer-model", {"path": "model/tokenizer.json"}),
            ],
        )
        
    ],
)

In [52]:
from vespa.package import Schema, Field

app_package.schema.add_fields(
            Field(name="id", type="string", indexing=["attribute", "summary"], primary_key=True),
            Field(name="url", type="string", indexing=["index", "summary"], index="enable-bm25"),
            Field(name="title", type="string", indexing=["index", "summary"], index="enable-bm25"),
            Field(name="creation_date", type="string", indexing=["attribute"]),
            Field(name="last_updated", type="string", indexing=["attribute"]),
            Field(name="paragraph_text", type="string", indexing=["index", "summary"], index="enable-bm25"),
            
            # intfloat/e5-small-v2
            Field(name="paragraph_text_embedding_384",
                  type="tensor<float>(x[384])",
                  indexing=["input paragraph_text", "embed e5-small-q", "index", "attribute"],
                  ann=HNSW(distance_metric="angular"),
                  is_document_field=False,),
      
            # intfloat/e5-large-v2
            # intfloat/multilingual-e5-large
            # thenlper/gte-large
            Field(name="paragraph_text_embedding_1024",
                  type="tensor<float>(x[1024])",
                  indexing=["index", "attribute"],
                  ann=HNSW(distance_metric="angular"),
                  ),
            # intfloat/e5-base-v2
            # intfloat/multilingual-e5-base
            # thenlper/gte-base
            Field(name="paragraph_text_embedding_768",
                  type="tensor<float>(x[768])",
                  indexing=["index", "attribute"],
                  ann=HNSW(distance_metric="angular"),
                  ),
            Field(name="time_stamp",
                type="long",
                indexing=[
                    "input creation_date",
                    "to_epoch_second",
                    "summary",
                    "attribute",
                ],
                is_document_field=False,),
)

In [101]:
app_package.schema.add_rank_profile(
    RankProfile(
        name="semantic_384",
        inputs=[("query(q)", "tensor<float>(x[384])")],
        inherits="default",
        first_phase="cos(distance(field,paragraph_text_embedding_384))",
    )
)


app_package.schema.add_rank_profile(
    RankProfile(
        name="semantic_768",
        inputs=[("query(q)", "tensor<float>(x[768])")],
        inherits="default",
        first_phase="cos(distance(field,paragraph_text_embedding_768))",
    )
)

app_package.schema.add_rank_profile(
    RankProfile(
        name="semantic_1024",
        inputs=[("query(q)", "tensor<float>(x[1024])")],
        inherits="default",
        first_phase="cos(distance(field,paragraph_text_embedding_1024))",
    )
)

app_package.schema.add_rank_profile(
    RankProfile(name="bm25", first_phase="2*bm25(title) + bm25(paragraph_text)")
)



app_package.schema.add_rank_profile(
    RankProfile(
        name="hybrid_384",
        inherits="semantic_384",
        first_phase=FirstPhaseRanking(
            expression="cos(distance(field,paragraph_text_embedding_384))"
        ),
        second_phase=SecondPhaseRanking(
            expression="firstPhase + log( bm25(title) + bm25(paragraph_text))+ freshness(time_stamp)"
        ),
        match_features=[
            "firstPhase",
            "bm25(title)",
            "freshness(time_stamp)",
            "secondPhase"
        ],
    )
)


app_package.schema.add_rank_profile(
    RankProfile(
        name="hybrid_768",
        inherits="semantic_768",
        first_phase=FirstPhaseRanking(
            expression="cos(distance(field,paragraph_text_embedding_768))"
        ),
        second_phase=SecondPhaseRanking(
            expression="firstPhase + log( bm25(title) + bm25(paragraph_text))+ freshness(time_stamp)"
        ),
        match_features=[
            "firstPhase",
            "bm25(title)",
            "freshness(time_stamp)",
            "secondPhase"
        ],
    )
)


app_package.schema.add_rank_profile(
    RankProfile(
        name="hybrid_1024",
        inherits="semantic_1024",
        first_phase=FirstPhaseRanking(
            expression="cos(distance(field,paragraph_text_embedding_1024))"
        ),
        second_phase=SecondPhaseRanking(
            expression="firstPhase + log( bm25(title) + bm25(paragraph_text))+ freshness(time_stamp)"
        ),
        match_features=[
            "firstPhase",
            "bm25(title)",
            "freshness(time_stamp)",
            "secondPhase"
        ],
    )
)




In [102]:
app_package.schema.add_field_set(
    FieldSet(name="default", fields=["title", "paragraph_text"])
)

In [103]:
Path("pkg").mkdir(parents=True, exist_ok=True)
app_package.to_files("pkg")

In [45]:
! mkdir -p pkg/model
! curl -L -o pkg/model/tokenizer.json \
  https://raw.githubusercontent.com/vespa-engine/sample-apps/master/simple-semantic-search/model/tokenizer.json

! curl -L -o pkg/model/e5-small-v2-int8.onnx \
  https://github.com/vespa-engine/sample-apps/raw/master/simple-semantic-search/model/e5-small-v2-int8.onnx
  
  
  

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  694k  100  694k    0     0  2072k      0 --:--:-- --:--:-- --:--:-- 2073k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 32.3M  100 32.3M    0     0  4363k      0  0:00:07  0:00:07 --:--:-- 5380k


In [152]:
from vespa.deployment import VespaDocker

vespa_docker = VespaDocker()
app = vespa_docker.deploy_from_disk(application_name="paragraph", application_root="pkg")

Waiting for configuration server, 0/60 seconds...
Waiting for configuration server, 5/60 seconds...
Waiting for application to come up, 0/300 seconds.
Waiting for application to come up, 5/300 seconds.
Waiting for application to come up, 10/300 seconds.
Waiting for application to come up, 15/300 seconds.
Waiting for application to come up, 20/300 seconds.
Waiting for application to come up, 25/300 seconds.
Using plain HTTP to connect to Vespa endpoint http://localhost:8080
Application is up!
Finished deployment.


In [ ]:
import requests

API_URL = "https://api-inference.huggingface.co/models/intfloat/multilingual-e5-large"
headers = {"Authorization": "Bearer "}

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()
	
output = query({
	"inputs": "Artificial Intelligence is rapidly evolving, with new advancements being made daily. The integration of AI in various sectors like healthcare, finance, and education is already transforming industries. In the coming years, AI's role is expected to expand, further influencing how we live and work.",
})





In [151]:
import requests

API_URL_384 = ""
API_URL_1024 = "https://api-inference.huggingface.co/models/intfloat/multilingual-e5-large"
API_URL_768 = ""
headers = {"Authorization": "Bearer "}  # Replace with your actual API key

def query(payload, api_url):
    response = requests.post(api_url, headers=headers, json=payload)
    return response.json()

input_text = "making algorithms more transparent"

# Query the API for 384, 1024, and 768 dimensional embeddings
output_384 = query({"inputs": input_text}, API_URL_384)
output_1024 = query({"inputs": input_text}, API_URL_1024)
output_768 = query({"inputs": input_text}, API_URL_768)



embedding_384 = output_384 if isinstance(output_384, list) else output_384.get('embedding', [])
embedding_1024 = output_1024 if isinstance(output_1024, list) else output_1024.get('embedding', [])
embedding_768 = output_768 if isinstance(output_768, list) else output_768.get('embedding', [])

# Check if the embeddings are the correct size
if len(embedding_384) == 384:
    print("Successfully retrieved 384-dimensional embedding.")
else:
    print(f"Unexpected embedding size: {len(embedding_384)} dimensions for 384.")

if len(embedding_1024) == 1024:
    print("Successfully retrieved 1024-dimensional embedding.")
else:
    print(f"Unexpected embedding size: {len(embedding_1024)} dimensions for 1024.")

if len(embedding_768) == 768:
    print("Successfully retrieved 768-dimensional embedding.")
else:
    print(f"Unexpected embedding size: {len(embedding_768)} dimensions for 768.")

jsonl_data = {
    "put": "id:your_namespace:document::1",
    "fields": {
        "id": "1",
        "url": "https://example.com/article1",
        "title": "The Future of Artificial Intelligence",
        "creation_date": "2023-01-01",
        "last_updated": "2023-01-02",
        "paragraph_text": input_text,
        "paragraph_text_embedding_384": {
            "assign": {
                "values": embedding_384
            }
        },
        "paragraph_text_embedding_1024": {
            "assign": {
                "values": embedding_1024
            }
        },
        "paragraph_text_embedding_768": {
            "assign": {
                "values": embedding_768
            }
        },
    }
}
print(jsonl_data)


Unexpected embedding size: 0 dimensions for 384.
Successfully retrieved 1024-dimensional embedding.
Unexpected embedding size: 0 dimensions for 768.
{'put': 'id:your_namespace:document::1', 'fields': {'id': '1', 'url': 'https://example.com/article1', 'title': 'The Future of Artificial Intelligence', 'creation_date': '2023-01-01', 'last_updated': '2023-01-02', 'paragraph_text': 'making algorithms more transparent', 'paragraph_text_embedding_384': {'assign': {'values': []}}, 'paragraph_text_embedding_1024': {'assign': {'values': [0.006448001600801945, -0.007938292808830738, -0.013899009674787521, -0.027674933895468712, 0.02352364920079708, -0.00887317955493927, -0.01350264623761177, 0.06610842794179916, 0.04185669869184494, -0.01662585325539112, 0.025726597756147385, -0.004236218985170126, -0.026108313351869583, -0.04116560146212578, -0.026077235117554665, -0.009734735824167728, -0.007455614395439625, 0.017334656789898872, 0.01262772735208273, -0.014369829557836056, 0.025386234745383263,

In [84]:
! vespa config set target local
! vespa feed articles.jsonl

{
  "feeder.operation.count": 2,
  "feeder.seconds": 0.559,
  "feeder.ok.count": 2,
  "feeder.ok.rate": 2.000,
  "feeder.error.count": 0,
  "feeder.inflight.count": 0,
  "http.request.count": 2,
  "http.request.bytes": 20809,
  "http.request.MBps": 0.021,
  "http.exception.count": 0,
  "http.response.count": 2,
  "http.response.bytes": 174,
  "http.response.MBps": 0.000,
  "http.response.error.count": 0,
  "http.response.latency.millis.min": 521,
  "http.response.latency.millis.avg": 521,
  "http.response.latency.millis.max": 521,
  "http.response.code.counts": {
    "200": 2
  }
}


In [161]:
result = app.query(
    body={
        "yql": 'select * from paragraph_content where {targetHits:10}nearestNeighbor(paragraph_text_embedding_384,q)',
        "input.query(q)": "embed(making algorithms more transparent)",
        "query": "making algorithms more transparent",
        "ranking.profile": "hybrid_384",
        "hits": 2,
    }
)
if len(result.hits) != 2:
    raise ValueError("Expected one hit, got {}".format(len(result.hits)))
print(json.dumps(result.hits, indent=4))

[
    {
        "id": "id:paragraph:paragraph::1",
        "relevance": "-Infinity",
        "source": "paragraph_content",
        "fields": {
            "matchfeatures": {
                "bm25(title)": 0.0,
                "firstPhase": 0.7900545788560296,
                "freshness(time_stamp)": 0.5945663580246914,
                "secondPhase": null
            },
            "sddocname": "paragraph",
            "documentid": "id:paragraph:paragraph::1",
            "time_stamp": 1724594400,
            "id": "1",
            "url": "https://example.com/article1",
            "title": "The Future of Artificial Intelligence",
            "paragraph_text": "Artificial Intelligence is rapidly evolving, with new advancements being made daily. The integration of AI in various sectors like healthcare, finance, and education is already transforming industries. In the coming years, AI's role is expected to expand, further influencing how we live and work."
        }
    },
    {
       

In [154]:
query_embedding=[0.006448001600801945, -0.007938292808830738, -0.013899009674787521, -0.027674933895468712, 0.02352364920079708, -0.00887317955493927, -0.01350264623761177, 0.06610842794179916, 0.04185669869184494, -0.01662585325539112, 0.025726597756147385, -0.004236218985170126, -0.026108313351869583, -0.04116560146212578, -0.026077235117554665, -0.009734735824167728, -0.007455614395439625, 0.017334656789898872, 0.01262772735208273, -0.014369829557836056, 0.025386234745383263, -0.020851854234933853, -0.056792810559272766, -0.05573813244700432, 0.003076172899454832, -0.0064931707456707954, -0.02400318905711174, -0.00638314476236701, -0.02305227890610695, -0.02726557105779648, 0.0032297580037266016, 0.003907620906829834, -0.045132115483284, -0.019383320584893227, -0.026892077177762985, 0.024853646755218506, 0.0337982140481472, 0.019976522773504257, -0.021120181307196617, -0.005183919332921505, -0.00476978300139308, 0.04508016258478165, -0.0013987767742946744, -0.04778396710753441, -0.007228809408843517, 0.03558231517672539, 0.03064618445932865, 0.0030622025951743126, -0.018750278279185295, 0.022064782679080963, 0.023084815591573715, 0.03359546139836311, -0.001103329355828464, -0.03933033347129822, -0.014573934487998486, 0.037265975028276443, -0.02597525715827942, 0.021365610882639885, -0.047678183764219284, -0.012905044481158257, -0.03206056356430054, -0.008452973328530788, -0.0010255142115056515, -0.02742517553269863, -0.024707337841391563, 0.009544520638883114, 0.02246377058327198, 0.012573403306305408, -0.037683527916669846, 0.029179830104112625, 0.0053383284248411655, 0.032592564821243286, -0.012160985730588436, -0.004311869386583567, 0.0012812338536605239, -0.018579591065645218, -0.0031565483659505844, -0.008188785053789616, 0.011590101756155491, -0.002188159618526697, 0.05824965611100197, -0.02344491146504879, 0.026361020281910896, -0.005210971459746361, 0.01938171498477459, 0.034083928912878036, 0.011877471581101418, 0.03938522934913635, 0.049441296607255936, 0.018025416880846024, -0.02572796307504177, 0.035906028002500534, 0.05155504494905472, -0.03907723352313042, -0.028488732874393463, -0.011973164044320583, 0.03561602160334587, 0.019334550946950912, 0.007997014559805393, -0.0005806165281683207, 0.007621073164045811, -0.009141860529780388, 0.005217357538640499, -0.03367874026298523, -0.013344524428248405, 0.02446119487285614, 0.011198341846466064, 0.05268936604261398, 0.0027651286218315363, -0.0005407718126662076, 0.004783721175044775, 0.025038480758666992, 0.03239338845014572, 0.00065911456476897, 2.106382089550607e-05, -0.010900168679654598, 0.006295515224337578, -0.012852770276367664, -0.020386656746268272, 0.01153983362019062, 0.05020647495985031, 0.013506176881492138, -0.0018935541156679392, -0.057326991111040115, 0.033003754913806915, -0.02025863341987133, 0.011176932603120804, 0.06511285901069641, -0.04856231063604355, 0.023789575323462486, 0.02176898904144764, 0.0233377143740654, -0.04817359894514084, 0.005476025864481926, -0.06564082950353622, -0.02040082961320877, -0.021450713276863098, -0.016981160268187523, -0.026332205161452293, -0.008498176001012325, 0.0013554139295592904, 0.03749930486083031, 0.016716787591576576, -0.047892455011606216, -0.01281684823334217, -0.02659052051603794, -0.04217872396111488, 0.006197559647262096, -0.00997555535286665, 0.01684136874973774, -0.0017496986547484994, -0.04436579346656799, -0.05745604634284973, 0.013329832814633846, 0.02659599296748638, -0.019271481782197952, 0.022360075265169144, 0.037922654300928116, -0.005473428871482611, 0.031024517491459846, -0.01642279513180256, -0.05258597061038017, -0.019749335944652557, -0.019926128908991814, 0.011263358406722546, -0.045787837356328964, 0.03766464442014694, 0.04497892037034035, 0.010876545682549477, -0.023799151182174683, -0.04588313400745392, -0.06022808700799942, 0.020586581900715828, -0.06067974120378494, -0.030913835391402245, 0.023624764755368233, 0.031001517549157143, 0.04746374860405922, 0.03506314381957054, -0.05068570375442505, 0.03818833455443382, 0.013376416638493538, 0.060571204870939255, -0.02656753733754158, -0.03257729113101959, 0.03144214302301407, 0.027350593358278275, 0.014653006568551064, 0.014780780300498009, 0.02487478218972683, 0.006857129279524088, -0.0351625457406044, 0.002021356485784054, -0.00970400683581829, -0.03770221024751663, 0.047257162630558014, 0.025922900065779686, 0.033509232103824615, -0.02860766276717186, -0.04757848381996155, 0.007949353195726871, 0.044761765748262405, -0.04532437399029732, 0.031489819288253784, 0.018912332132458687, -0.023826466873288155, -0.016283994540572166, -0.04099085181951523, 0.01773339882493019, -0.041242338716983795, -0.031162872910499573, -0.009513209573924541, 0.04511088877916336, 0.008135063573718071, -0.032312121242284775, -0.03698630630970001, -0.03299105912446976, -0.010723463259637356, 0.022178754210472107, -0.026110533624887466, 0.0007704575546085835, 0.057349398732185364, -0.0012221282813698053, 0.02330641821026802, 0.02945399098098278, 0.012286935932934284, 0.03945924714207649, 0.05128905922174454, 0.06695757806301117, 0.001994612393900752, -0.007043563760817051, 0.024127962067723274, -0.0699395164847374, -0.021989645436406136, 0.000933798321057111, -0.03253621980547905, -0.029908953234553337, -0.003438974032178521, 0.030870633199810982, 0.032889414578676224, -0.034633345901966095, 0.022605009377002716, 0.01759393885731697, -0.01287788338959217, -0.005981347989290953, -0.005064104683697224, 0.021025637164711952, -0.01045188494026661, 0.013825868256390095, -0.03560084477066994, 0.056814223527908325, 0.02268766425549984, -0.034184593707323074, 0.005199981853365898, 0.056445568799972534, 0.03493506461381912, 0.03489360958337784, -0.014281798154115677, 0.00593793299049139, -0.022553713992238045, 0.021138306707143784, -0.005017693620175123, -0.009640639647841454, 0.03359280526638031, 0.0448596253991127, -0.0019280354026705027, -0.02108059450984001, -0.04882896691560745, -0.021522246301174164, -0.03205093741416931, -0.04458010569214821, -0.023245876654982567, -0.02827712520956993, -0.032324835658073425, -0.03728451579809189, 0.021923044696450233, -0.028675228357315063, -0.014790241606533527, 0.022318247705698013, -0.00224233977496624, -0.011308923363685608, -0.011221645399928093, 0.024432985112071037, -0.02059873938560486, 0.024627428501844406, 0.002506683813408017, 0.013266563415527344, -0.008462422527372837, 0.012594823725521564, -0.015073317103087902, 0.029226969927549362, -0.00258062407374382, 0.09665205329656601, 0.011162536218762398, 0.033579930663108826, 0.03282182663679123, -0.03776778280735016, -0.09745369106531143, -0.055932529270648956, 0.010957826860249043, -0.00020279892487451434, -0.03698292747139931, 0.017386922612786293, 0.03391418606042862, -0.02449028007686138, -0.009503837674856186, -0.004597994964569807, 0.002704509301111102, -0.07765045762062073, 7.732178346486762e-05, 0.03161204606294632, 0.010779191739857197, 0.025327056646347046, -0.025574035942554474, -0.06217076629400253, -0.030870111659169197, -0.02579774148762226, 0.01945476420223713, -0.027097051963210106, 0.062239035964012146, -0.05094299837946892, -0.003953807987272739, -0.030692318454384804, -0.0036048979964107275, 0.07400745898485184, -0.0059636440128088, 0.02787446603178978, 0.001960625872015953, 0.006060945335775614, -0.03490718826651573, 0.030583208426833153, -0.03935953974723816, -0.009426087141036987, 0.03068338707089424, -0.007739086169749498, 0.00041721787420101464, -0.05807938426733017, 0.0227822195738554, 0.02373308502137661, -0.021537769585847855, -0.0018496820703148842, -0.02025572583079338, -0.024889452382922173, -0.003371587023139, 0.020557111129164696, -0.0710398331284523, 0.031598009169101715, -0.024142572656273842, 0.045136403292417526, -0.004327045287936926, 0.00900624506175518, -0.06158562749624252, 0.0068589430302381516, -0.006138273980468512, -0.02628323808312416, 0.03576602786779404, 0.04675791412591934, -0.04022032767534256, 0.04290463402867317, -0.01519053429365158, 0.03205135092139244, 0.014256672002375126, 0.04775282368063927, -0.012356401421129704, -0.016530442982912064, -0.0021951748058199883, 0.009789147414267063, -0.0141460495069623, -0.03802597150206566, -0.03279200568795204, 0.026726819574832916, 0.008659006096422672, -0.035838447511196136, 0.006432848516851664, 0.02803480066359043, -0.01007795613259077, -0.0629899799823761, -0.020616315305233, 0.0063428571447730064, -0.06181147322058678, -0.031114060431718826, -0.03685445338487625, 0.0118617108091712, -0.027808403596282005, 0.019441673532128334, -0.034088779240846634, 0.16624031960964203, 0.034731876105070114, 0.04882786422967911, -0.016925811767578125, 0.004452391527593136, 0.022092504426836967, 0.04326038435101509, 0.05227941274642944, 0.01993868127465248, 0.04471220076084137, -0.017418205738067627, 0.0008419292862527072, 0.0204930417239666, -0.03274202719330788, 0.053208865225315094, 0.03067167103290558, 0.01554642803966999, 0.04287572205066681, 0.056437697261571884, 0.030842551961541176, 0.04707322269678116, -0.016078365966677666, 0.06163070723414421, 0.04973989352583885, -0.07255570590496063, -0.044600822031497955, -0.009167300537228584, 0.06161113828420639, -0.031141890212893486, 0.026695141568779945, -0.030596300959587097, 0.0529337152838707, -0.017544325441122055, -0.0021580508910119534, 0.027600958943367004, 0.03038201667368412, 0.021268313750624657, -0.01154597382992506, 0.05751695856451988, -0.015258435159921646, -0.029420703649520874, 0.01448085531592369, -0.053935084491968155, 0.05827200785279274, -0.005859040655195713, 0.028885386884212494, 0.06265271455049515, -0.02724972553551197, -0.035094939172267914, -0.007225418463349342, 0.025728587061166763, 0.04425180330872536, -0.036906685680150986, -0.03591470420360565, -0.01417494472116232, -0.03863148018717766, -0.021219907328486443, -0.01944129727780819, 0.014915946871042252, 0.00984449964016676, 0.0404769666492939, -0.026890167966485023, -0.012120043858885765, -0.008845572359859943, 0.025580110028386116, 0.033282604068517685, -0.0320059210062027, -0.0272835586220026, -0.049985382705926895, -0.007812700234353542, -0.02709491364657879, -0.03816662356257439, -0.018828975036740303, 0.003926449920982122, 0.02370472066104412, -0.01823130063712597, -0.007154621183872223, 0.07064724713563919, 0.005009579937905073, -0.021325377747416496, 0.04292767494916916, -0.016468442976474762, 0.0066060093231499195, 0.0500493161380291, 0.052943356335163116, 0.004691548645496368, 0.0034994075540453196, 0.0041478294879198074, -0.03365258127450943, 0.03817480430006981, 0.01183458510786295, 0.0007161847315728664, -0.05033392086625099, 0.027499541640281677, 0.04474073648452759, -0.04442187398672104, 0.02108345925807953, 0.001204482396133244, -0.03983174264431, -0.025233227759599686, -0.017461955547332764, 0.005686959717422724, -0.034983016550540924, -0.0464804470539093, -0.020079059526324272, -0.01256524957716465, 0.01625829003751278, 0.04588517174124718, 0.03287413716316223, -0.008935270830988884, 0.004988348111510277, -0.0006415985990315676, 0.01304432563483715, -0.05693920701742172, 0.0321647934615612, 0.032704319804906845, 0.02426089532673359, -0.04576234519481659, 0.005681989248842001, 0.025409361347556114, -0.03917092829942703, 0.07160820811986923, -0.02437017671763897, 0.06830520927906036, -0.014969264157116413, 0.046124763786792755, -0.027655186131596565, 0.0028126079123467207, 0.023651469498872757, 0.009650171734392643, 0.031196488067507744, -0.014393895864486694, -0.02136281132698059, 0.02867848426103592, 0.0082606952637434, 0.003599512856453657, 0.04264960438013077, -0.02987947314977646, 0.038942597806453705, 0.034428611397743225, -0.0028712896164506674, 0.035933058708906174, -0.02381531335413456, 0.005797651130706072, 0.015508901327848434, 0.014003746211528778, -0.007518419995903969, -0.049704138189554214, -0.032862503081560135, -0.030102185904979706, 0.027198880910873413, -0.0403452143073082, 0.011188596487045288, 0.02116488479077816, -0.0015574332792311907, -0.045918289572000504, 0.018204355612397194, 0.030751027166843414, 0.008181121200323105, -0.04471508041024208, -0.02442583255469799, -0.017420204356312752, -0.004826666321605444, 0.0746200680732727, 0.03721682354807854, 0.03847017139196396, -0.026027584448456764, 0.03336549177765846, 0.039570070803165436, -0.008976176381111145, -0.014413140714168549, -0.03162238001823425, -0.013447988778352737, 0.028435751795768738, 0.028029723092913628, -0.032746609300374985, -0.049197006970644, -0.0010590668534860015, -0.012806383892893791, 0.011311418376863003, -0.039354123175144196, 0.022920621559023857, 0.02561018243432045, -0.032339174300432205, -0.023803848773241043, -0.019027959555387497, 0.001352441729977727, -0.029547102749347687, -0.005764626897871494, 0.00846149679273367, -0.03398970887064934, 0.06960850208997726, -0.015728719532489777, -0.00890449807047844, -0.047550078481435776, 0.010789208114147186, 0.016113579273223877, 0.014317721128463745, -0.014402608387172222, -0.04051365703344345, 0.012942359782755375, -0.01601303741335869, -0.008598721586167812, -0.017601147294044495, 0.002577555598691106, -0.017876822501420975, 0.007212350610643625, 0.017264781519770622, 0.02950095944106579, 0.004864492919296026, -0.020829064771533012, -0.028838379308581352, -0.013570135459303856, 0.0108908386901021, 0.015572830103337765, 0.03283201530575752, 0.0005131022189743817, 0.023981431499123573, 0.030418580397963524, -0.005952433682978153, 0.024401716887950897, 0.03873571380972862, 0.01621285080909729, -0.0406801737844944, -0.06005275994539261, -0.01441227737814188, -0.006843110080808401, -0.00172846217174083, -0.012692904099822044, -0.05705401673913002, 0.006899978965520859, -0.005146957002580166, -0.03637659177184105, -0.03259111940860748, -0.01873553916811943, -0.006524998694658279, 0.03233223780989647, 0.02899615466594696, -0.014071729965507984, -0.04902574047446251, 0.04988846555352211, -0.006371368188410997, 0.06277129799127579, -0.033227354288101196, -0.02109619230031967, 0.012387970462441444, -0.04537408426403999, -0.003559040604159236, 0.025483541190624237, 0.03526538237929344, 0.006610925309360027, -0.03414571285247803, 0.02815837971866131, 0.03731880709528923, -0.01631198078393936, -0.04359866678714752, -0.00850497093051672, -0.023279540240764618, -0.04828894883394241, -0.034934304654598236, 0.01443701982498169, 0.015069952234625816, -0.0482664592564106, 0.0072333007119596004, -0.040532127022743225, 0.06137378513813019, 0.046537671238183975, -0.01970655284821987, -0.011711479164659977, -0.051356393843889236, 0.011836318299174309, -0.009092044085264206, -0.028254641219973564, -0.06763743609189987, 0.07063878327608109, -0.007008575368672609, -0.044382959604263306, -0.020315514877438545, -0.02894599549472332, -0.021709995344281197, -0.004023036453872919, 0.049231015145778656, -0.04801446571946144, 0.014341182075440884, 0.024472463876008987, 0.03517310321331024, -0.029876327142119408, 0.03667067363858223, 0.018369819968938828, -0.04652855545282364, -0.014348343946039677, 0.04376181960105896, -0.015085737220942974, 0.043858252465724945, -0.03101743571460247, -0.02566840872168541, -0.060186125338077545, -0.0010107773123309016, -0.0027807732112705708, -0.02718602865934372, 0.012733349576592445, -0.04081907495856285, -0.049241188913583755, -0.015528541058301926, -0.019896406680345535, 0.005968044977635145, -0.039225976914167404, -0.04340799152851105, 0.04156670719385147, 0.019914457574486732, -0.017183924093842506, -0.057794470340013504, 0.01540242601186037, 0.021518714725971222, -0.02672269009053707, 0.013371025212109089, 0.0083487993106246, -0.030798254534602165, -0.03098841942846775, 0.042928315699100494, -0.037201132625341415, -0.01184134278446436, -0.0055295792408287525, -0.031125672161579132, 0.019693877547979355, 0.004176526330411434, -0.012984821572899818, 0.03187791630625725, 0.015554307028651237, -0.0036323147360235453, 0.008933973498642445, -0.01074180006980896, -0.03331417590379715, 0.007199989631772041, -0.017473287880420685, -0.051948897540569305, -0.03904547169804573, -0.016273632645606995, -0.011545038782060146, -0.042214035987854004, -0.04244426637887955, 0.025030560791492462, 0.06272488087415695, -0.03517305105924606, 0.04444599151611328, -0.017226703464984894, 0.002272873418405652, -0.008460900746285915, -0.006382892373949289, -0.034113164991140366, -0.029478685930371284, 0.04023473337292671, -0.03636769950389862, 0.017406968399882317, -0.035532742738723755, 0.018482089042663574, -0.051569387316703796, 0.013443299569189548, 0.025738583877682686, -0.021350698545575142, -0.02782622165977955, 0.009989764541387558, 0.0023135393857955933, -0.03235742822289467, -0.01962912082672119, 0.019729025661945343, -0.03872336819767952, 0.005289433989673853, -0.06981714814901352, 0.010217786766588688, -0.03765498846769333, 0.024440420791506767, -0.029796212911605835, -0.017923086881637573, -0.03240388631820679, 0.031881220638751984, -0.05825630947947502, 0.01801714301109314, 0.023957254365086555, -0.023569663986563683, 0.0058869714848697186, -0.03651059791445732, -0.02642054855823517, -0.0003744913265109062, 0.020146271213889122, -0.027508849278092384, -0.00731432531028986, 0.0028781318105757236, 0.019857218489050865, 0.026569874957203865, -0.029560869559645653, 0.04856770485639572, -0.029477830976247787, -0.0055799768306314945, -0.07109896838665009, 0.001488352776505053, -0.005293012131005526, -0.0055383192375302315, -0.03760723024606705, -0.017574291676282883, -0.05728292837738991, 0.0018570942338556051, 0.03271416947245598, -0.05014878138899803, 0.02527746930718422, 0.012955985963344574, 0.03121144510805607, -0.012605439871549606, -0.034245677292346954, 0.009087675251066685, 0.05078544840216637, 0.004531365819275379, -0.011749796569347382, 0.024358335882425308, 0.017063722014427185, 0.0029990694019943476, -0.01192022766917944, 0.019794678315520287, 0.037530265748500824, 0.04367399215698242, 0.041354700922966, 0.01568600721657276, -0.006443540100008249, -0.03435663878917694, -0.00594161543995142, -0.005329194013029337, 0.02441590465605259, 0.04442359507083893, -0.0612279549241066, 0.0020510246977210045, -0.038632508367300034, -0.04377681016921997, -0.0302225723862648, 0.022321229800581932, -0.0398448146879673, 0.030970152467489243, -0.058321330696344376, 0.046653181314468384, -0.001336250570602715, 0.04931388795375824, 0.006701492238789797, -0.02274845354259014, 0.004604254383593798, -0.030972925946116447, -0.020573828369379044, -0.022341670468449593, 0.023582153022289276, 0.03997449949383736, 0.020477941259741783, 0.022547531872987747, 0.04137378931045532, 0.0026439735665917397, 0.02591940574347973, 0.049101557582616806, 0.0030366661958396435, -0.020326752215623856, -0.05000950023531914, 0.0023511347826570272, -0.035079047083854675, -0.011480347253382206, -0.0862463191151619, -0.013493181206285954, -0.013275068253278732, 0.039442241191864014, -0.007924293167889118, -0.026812314987182617, -0.014300835318863392, -0.016573045402765274, 0.03150840476155281, -0.011072760447859764, -0.05591041222214699, 0.014674415811896324, 0.0212435033172369, 0.010165208019316196, -0.001712742610834539, -0.0490485318005085, -0.004032712429761887, 0.013620099984109402, -0.0245162695646286, 0.014143883250653744, -0.009617974981665611, -0.01709008403122425, -0.04430796205997467, 0.01664506085216999, -0.04004629701375961, 0.03263856843113899, 0.03632243350148201, -0.0349847674369812, -0.016261287033557892, -0.020692428573966026, -0.05107581615447998, 0.002297057071700692, -0.02134731225669384, 0.007449484895914793, -0.03991904854774475, 0.003979406785219908, 0.034779660403728485, 0.034156475216150284, -0.01653323695063591, -7.010503759374842e-05, 0.0653347596526146, -0.008945630863308907, 0.0638238936662674, 0.0047864909283816814, -0.054757241159677505, -0.012103034183382988, -0.008578761480748653, -0.007711517158895731, -0.04504426196217537, -0.006670613773167133, -0.018647251650691032, 0.0044670747593045235, -0.020450789481401443, 0.02108156867325306, -0.023996923118829727, -0.03036131151020527, 0.06541658192873001, -0.016947509720921516, 0.031398955732584, -0.0094077680259943, 0.057388272136449814, -0.0343046598136425, 0.013858937658369541, -0.048695117235183716, 0.05839107558131218, -0.05541921779513359, 0.04160193353891373, 0.010506863705813885, 0.004065503366291523, -0.01222182996571064, -0.020180581137537956, 0.09759955108165741, -0.024938834831118584, 0.06117482855916023, -0.02309337817132473, -0.019302265718579292, -0.03681807592511177, -0.02545914053916931, -0.017905686050653458, 0.02586103230714798, -0.018980221822857857, -0.021965954452753067, -0.005287013482302427, -0.02419484779238701, 0.024298515170812607, -0.011178260669112206, -0.030724886804819107, 0.03319171816110611, 0.026544542983174324, 0.0024102262686938047, 0.0027444486040621996, 0.0017516494262963533, 0.018579093739390373, 0.03671460598707199, -0.0039421687833964825, 0.008198902942240238, -0.0301873367279768, 0.03641368821263313, 0.005646207369863987, -0.04931429401040077, 0.039921145886182785, 0.0036044144071638584, -0.004046980757266283, 0.003308740444481373, -0.006176423747092485, 0.04102259874343872, -0.033856723457574844, 0.034770701080560684, 0.037344858050346375, -0.006088803056627512, 0.05671430006623268, 0.05670866370201111, 0.04249807819724083, -0.00025451398687437177, 0.03316742926836014, -0.012301022186875343, -0.01681756041944027, 0.03190917894244194, 0.042680203914642334, 0.03811325877904892, 0.004508873447775841, 0.01983412355184555, -0.001986544579267502, 0.036613378673791885, -0.001964001916348934, 0.01376281026750803, -0.03512994199991226, 0.03260369598865509, 0.030118998140096664, -0.01824144460260868, -0.004484615288674831, 0.012237745337188244, -0.02791513316333294, 0.025444231927394867, -0.00044352657278068364, -0.028103066608309746, 0.03350356966257095, -0.00040445741615258157, -0.025312956422567368, -0.05657791718840599, -0.03610285371541977, -0.021716803312301636, -0.03489472717046738, 0.03279636800289154, 0.016566457226872444, -0.0012014767853543162, -0.024290194734930992, 0.03085724450647831, 0.04659583047032356, 0.010981136932969093, 0.05090998858213425, -0.02932330034673214, 0.007074779365211725, 0.010149153880774975, 0.03232698142528534, 0.022428803145885468, -0.006623417139053345, -0.0056697288528084755, -0.04000231996178627, 0.05157289654016495, -0.035598237067461014, 0.034122664481401443, -0.04487551376223564, -0.014970909804105759, 0.017283717170357704, 0.01591739058494568, 0.017106855288147926, 0.023073723539710045, -0.041608650237321854, -0.03876453638076782, 0.04271393641829491, -0.025828057900071144, 0.008466544561088085, 0.032270945608615875, 0.029756421223282814, -0.009613888338208199, -0.00602441793307662, -0.005231174174696207]

In [157]:
len(query_embedding)

1024

In [158]:
result = app.query(
    body={
        "yql": 'select * from paragraph_content where ({targetHits:1}nearestNeighbor(paragraph_text_embedding_1024,q))',
        "input.query(q)": query_embedding,
        "query": "making algorithms more transparent",
        "ranking.profile": "hybrid_1024",
        "hits": 2,
    }
)
if len(result.hits) != 1:
    raise ValueError("Expected one hit, got {}".format(len(result.hits)))
print(json.dumps(result.hits, indent=4))

[
    {
        "id": "id:paragraphs:paragraph::2",
        "relevance": "-Infinity",
        "source": "paragraph_content",
        "fields": {
            "matchfeatures": {
                "bm25(title)": 0.0,
                "firstPhase": 0.8466507701069591,
                "freshness(time_stamp)": 0.0,
                "secondPhase": null
            },
            "sddocname": "paragraph",
            "documentid": "id:paragraphs:paragraph::2",
            "time_stamp": 1661436000,
            "id": "2",
            "url": "https://example.com/article2",
            "title": "Challenges in Machine Learning",
            "paragraph_text": "Machine Learning faces many challenges despite its successes. Data privacy, bias in algorithms, and the high computational cost of training models are significant hurdles. Researchers are focusing on making algorithms more transparent and interpretable, while also improving the efficiency of training processes."
        }
    }
]


In [148]:
vespa_docker.container.stop()

In [132]:
vespa_docker.container.remove() #rmi image